In [9]:
DATAPATH = "/mnt/cloud/lithuanian-asr/data"

In [1]:
!pip install pympi-ling

In [2]:
import pympi
from pathlib import Path
from pydub import AudioSegment

In [15]:
def get_eaf_data(filename):
    filepath = Path(filename)
    base = filepath.stem
    eaf = pympi.Elan.Eaf(filename)
    tiers = []

    def is_simple_layout(tiers):
        if len(tiers) == 1 and 'speech' in tiers:
            return True
        elif len(tiers) == 2 and 'speech' in tiers and 'noise' in tiers:
            return True
        elif len(tiers) == 3 and 'speech' in tiers and 'noise' in tiers and 'ss' in tiers:
            return True
        else:
            return False
    if is_simple_layout(eaf.tiers):
        tiernames = ['speech']
        simple = True
    else:
        skip = ['noise', 'ss']
        tiernames = [a for a in eaf.tiers.key() if a not in skip]
        simple = False

    for tiername in tiernames:
        for tier in eaf.tiers[tiername][0].keys():
            current = {}
            id = f"{base}_{tier}"
            if not simple:
                id = f"{base}_{tiername}_{tier}"
            current["id"] = id
            data = eaf.tiers['speech'][0][tier]
            current["start"] = eaf.timeslots[data[0]]
            current["end"] = eaf.timeslots[data[1]]
            current["text"] = data[2]
            tiers.append(current)

    return tiers

In [3]:
def write_split_wavs(outdir, filename, data):
    outpath = Path(outdir)
    original = AudioSegment.from_wav(filename)
    for piece in data:
        outfile = outpath / f"{piece['id']}.wav"
        audio = original[piece["start"]:piece["end"]]
        audio.export(str(outfile), format="wav")


In [13]:
def append_to_tsv(tsv_file, data):
    with open(tsv_file, "a") as f:
        for item in data:
            f.write(f"{item['id']}\t{item['text']}\n")

In [ ]:
def is_simple_layout(tiers):
    if len(tiers) == 1 and 'speech' in tiers:
        return True
    elif len(tiers) == 2 and 'speech' in tiers and 'noise' in tiers:
        return True
    else:
        return False

In [6]:
SAMPLE = "/mnt/cloud/lithuanian-asr/data/R_RS_F3_AS113_01.eaf"
SAMPLE_WAV = SAMPLE.replace(".eaf", ".wav")
data = get_eaf_data(SAMPLE)
write_split_wavs("/tmp/foo", SAMPLE_WAV, data)

In [16]:
for eaf_file in Path(DATAPATH).glob("*.eaf"):
    data = get_eaf_data(eaf_file)
    wav_file = str(eaf_file).replace(".eaf", ".wav")
    write_split_wavs("/mnt/cloud/liepa-split", wav_file, data)
    append_to_tsv("/mnt/cloud/liepa-split/text.tsv", data)

Error reading file /mnt/cloud/lithuanian-asr/data/X_MS001_01.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_IS007_01.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_VP018_01.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_IS004_01.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_IS041_01.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_VP013_01.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_IM010_02.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_IM009_01.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_VP009_01.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_VP020_01.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_IS023_01.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_IM013_02.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_IM005_01.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_MS011_01.eaf
Error reading file /mnt/cloud/lithuanian-asr/data/X_VP012_01.eaf
Error reading file /mnt/c